# ENI Port Scan

Use this runbook to diagnose the source of a port scan on an AWS ENI

## Parameters
* **eni_id**: The ENI ID of the network adapters receiving the port scan.
* **account_id**: THe AWS account that the port scan is occurring.

In [ ]:
from jupyterirtools import sso

account_id = "251344881676"
eni_id = "eni-04050d5ac94cd243b"

sso.login("Jupyter-IR-ViewOnly", account_id)

In [ ]:
from jupyterirtools import athena
from datetime import date
from datetime import timedelta

import importlib
importlib.reload(athena)

today = date.today()
yesterday = today - timedelta(days = 1)
    
sql = f"""SELECT srcaddr, srcport, dstaddr, dstport, count(packets) num_packets, sum(packets) packet_count, sum(bytes) sum_bytes
FROM "${{database_name}}"."vpc_flow_logs" 
WHERE "timestamp" >= '{str(yesterday.year).zfill(2)}/{str(yesterday.month).zfill(2)}/{str(yesterday.day).zfill(2)}'
AND accountid = '{account_id}' AND interface_id = '{eni_id}'
GROUP BY interface_id, srcaddr, srcport, dstaddr, dstport
ORDER BY sum(bytes) DESC
limit 1000;"""

#print(sql)
display(athena.run_query(sql))

# Recommended Actions
* Determine if IPs are internal or external.
* Perform a reputation lookup for the most frequently accessed external IPs. 
* Is a WAF rule needed to block the IP, or adjustments to the rate throttles.